# Application Programming Interface (API) von LLMs

Dieses Notebook demonstriert den programmatischen Umgang mit KI modellen. Mit einer API können Programme und Systeme selbstständig mit einem LLM kommunizieren, ohne dass ein Mensch manuell eine Eingabe über ein User Interface (UI) machen muss. Wir werden uns die API on OpenAI (die Firma, die ChatGPT entwickelt hat) als Beispiel ansehen, denn zum einen ist sie sehr gut dokumentiert (siehe [hier](https://platform.openai.com/docs/api-reference/)), das heisst man kann Funktionsweisen schnell und bequem nachlesen. Zum anderen folgen die meisten grossen AI Modelle einer ähnlichen Struktur.

Dieses Notebook ist wiefolgt aufgebaut:
- API Key
    - Kurze Erklärung was ein API Key ist und warum wir ihn brauchen (für dieses Notebook ist kein Key nötig)
- OpenAI Bibliothek: Installation
    - Installation der Python Bibliothek von OpenAI
- Chat Completion Endpoint
    - Wie schreibe ich einen API Call an ChatGPT in python
    - Aufgabe 1
- How to Receive a Response
    - Wie verarbeite ich die Antwort von GPT in python weiter?
    - Aufgabe 2
- Programmatisches Erstellen der Inputs
    - Ein Skript schreiben, dass verschiedene Prompts automatisch abfragt
    - Aufgabe 3
- Structured Outputs
    - Wie kann ich das LLM dazu bringen mir JSON output zu geben, den ich programmatisch weiterverarbeiten kann
    - Aufgabe 4

## API Key



Moderne KI-Modelle sind sehr gross und brauchen nicht nur viel Speicherplatz, sondern auch viel Rechenleistung. Deswegen ist es sinnvoll, wenn sie auf grossen Servern laufen (wie diese von OpenAI) und die User mit Requests, diese Server abfragen können. Das passiert, wenn wir über Unseren Browser das UI von ChatGPT aufrufen. Wie im UI brauchen wir ein log-in, was durch einen API-key geschieht.<br><br>
Für dieses Notebook ist kein API-Key nötig, denn es verwendet Beispiel Inputs und Outputs. Falls ihr euch interessiert ein Sprachmodell für eure Projektarbeit zu verwenden braucht ihr einen solchen Key, besprecht das mit der/dem Betreuer:in eurer Arbeit.



### Exkurs: Pricing von APIs für LLMs

Dieser Abschnitt erklärt wie sich die Kosten von der Nutzung von APIs für LLMs berechnen. Kurz zusammengefasst, solange ihr nicht riesige Datenmengen verarbeitet sind die Kosten minimal. Dieser Teil ist nur bei Interesse zu lesen und ist für den Rest des Notebooks nicht relevant. Ihr könnt also getrost zum Abschnitt *OpenAI Bibliothek - Installation* springen.

Die meisten APIs der grossen Modelle sind kostenpflichtig, das heisst man zahlt für jeden Request. 
Der Umgang mit API-Keys bedarf etwas vorsicht, da es im Grunde nichts anderes als eine lange Zeichenkette ist. Jeder im Besitz dieser Zeichenkette kann Requests in eurem Namen machen.



Die Kosten für die Abfrage von LLM APIs werden pro Token angegeben, so sehen wir zum Beispiel auf der [Pricing liste von OpenAI](https://openai.com/api/pricing/) Folgendes:![gpt-4 Pricing](gpt-4_pricing.png " ")

Was ist ein *Token*?

Grosse LLMs teilen Texte nicht in Wörter ein sondern in Byte-Pair Encoded (BPE) Tokens. Ein Text hat typischerweise mehr Tokens als Wörter, aber es gibt Aussnahmen. Je weiter eine Sprache vom Englischen entfernt ist, desto mehr Tokens braucht es (im Durchschnitt) um die gleiche Anzahl Wörter darzustellen.

Warum *Tokens*?

Ein Modell, das das gesamte Vokabular einer Sprache hat würde sehr aufwändig sein, vorallem wenn es multilingual sein soll. Ein Modell, dass nur Buchstabe für Buchstabe liest kann den Kontext nur schwer erfassen, ausserdem muss es verschiedene Schriftarten beherrschen. Die Lösung ist das sogenannte Byte-Pair Encoding, indem das Modell für sich selbst sinnvolle "Wortgrenzen" macht. Hier ist "Wortgrenzen" in Anführungszeichen, da dies auf dem level der Bytes geschieht (also die Reihe von 0 und 1, die ein Programm verarbeitet). Das gibt einen guten Kompromiss. Die Tokens eines LLMs machen manchmal Sinn für Menschen, zum Beispiel ist das Suffix "en" oft eines. Mit diesem können viele weitere Wörter gebildet werden. Es muss aber nicht unbedingt linguistische Hintergründe haben, wenn es für das modell Sinn macht.

Diese Website https://gpt-tokenizer.dev/ erlaubt es Euch, für verschiedene GPT Modelle einen Text einzugeben und euch die Anzahl Tokens anzeigen zu lassen. Schauen wir uns ein Beispiel an:


![example_sentence](example_sentence.png)
![beispielsatz](beispielsatz.PNG)



Wir sehen, dass das deutsche Wort "Beispielsatz" von dem Modell mit 5 Tokens dargestellt wird, während alle englischen Wörter in diesem Beispiel einen eigenen Token bekommen. Eine weitere Eigenheit wird sichtbar: Das Modell sieht Leerzeichen vor oder nach den Wörtern meist als zugehörend zu dem Token, während der Punkt als Satzzeichen alleinstehend ist.



#### Konklusion

Wenn wir uns wieder der Abbildung des Pricing von Openai widmen, sehen wir, dass die Preise zum einen aufgeteilt sind in input und output (cached muss uns jetzt nicht weiter interessieren). Ausserdem sind sie angegeben pro Millionen Tokens. Das heisst eine Prompt von 200 Tokens, die eine Antwort von 500 Tokens erhält kostet $0.0055 mit GPT-4 und mit GPT-4-mini (was für die meisten Anwendungen fast gleichwertig performt) $0.00033.

Kleinere Experimente mit der API sind also gut zu machen, ohne grosse Summen zu bezahlen. Bevor man sich daran macht, ein grösseres Datenset mit einem LLM zu bearbeiten, ist jedoch eine Kostenschätzung sinnvoll.

## OpenAI Bibliothek - Installation

OpenAI hat eine Python-Bibliothek veröffentlicht, die es uns erlaubt mit python code die API abzurufen. Die meisten LLMs funktionieren ähnlich und viele sind sogar kompatibel mit OpenAI's Python Package. <br><br> Diese Bibliothek müssen wir installieren. Die folgende Code-Zelle tut dies aus dem Notebook heraus. Das %-symbol sagt dem Jupyter Kernel, dass der folgende Command im terminal und im Python environment, das gerade aktiv ist, auszuführen ist.
-  Nach erfolgreicher Installation **restartet das Kernel** indem ihr oben (unter dem Dateinamen) auf "Restart" klickt.
-  **Ihr braucht diese Zelle nur einmal auszuführen**, danach sollte das Package installiert sein, wenn ihr dieses Notebook mit demselben Kernel öffnet.

In [1]:
%pip install openai


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Nach erfolgreicher Installation und Restart, sollten wir die OpenAI bibliothek benutzen können. Folgende Code-Zeile sollte ohne Fehler laufen. Ihr braucht an dieser Stelle keinen API-Key einzufügen.

In [1]:
from openai import OpenAI

# Falls ihr einen API-Key habt, könnt ihr ihn hier eingeben. Passt auf, dass ihr ihn nicht ausversehen "publiziert", zum Beispiel auf Github.

API_KEY = "YOUR_API_KEY_HERE" 

# Iniziieren des Clients
client = OpenAI(api_key=API_KEY)

## Chat Completion Endpoint
In der folgenden Zelle ist ein Beispiel, wie man die API verwendet um ChatGPT abzufragen. APIs haben verschiedene **Endpoints**. Man kann sich das wie eine Tür zu einem bestimmten Service vorstellen:
- Die API ist das ganze Gebäude (z. B. OpenAI API).
- Ein Endpoint ist eine bestimmte Tür im Gebäude (z. B. die Tür für Textgenerierung).

Ihr müsst hier nicht alles verstehen, wir werden im Folgenden genauer darauf eingehen. Für jetzt genügt es, dass wir den API-Endpoint für ```chat.completion``` mit der ```create``` Methode abfragen. <br><br>
Wir haben bis jetzt keine gültigen API-Key angegeben, das heisst wir erwarten einen `AuthenticationError` bei der Ausführung, denn der Server weiss nicht wer wir sind und wird uns nicht "bedienen". Für diesen Fall habe ich ein kleines python objekt erstellt, das sich wie eine Antwort vom Server verhält.

In [ ]:
from openai import AuthenticationError  # Der Error, den wir abfangen wollen
from helper import mock_response  # Ein Objekt, das ich erstellt habe als Beispiel, solange wir keinen API-key verwenden

try:  # solange kein Error entsteht tue dies

  # Abfragen des Servers und die Antwort des Servers in der Variable 'response' speichern
  response = client.chat.completions.create(  
    model="gpt-4o-mini",
    messages=[
      {"role": "developer", "content": "You are a helpful assistant."},
      {"role": "user", "content": "Hello!"}
    ]
  )
  # Die Antwort aus der Response herausholen (mehr dazu siehe Unten)
  print(response.choices[0].message.content)
# Abfangen, wenn es einen AuthenticationError gibt und eine "mock-response" verwenden
except AuthenticationError as e:
  # Die Antwort aus der Mock-Response herausholen (mehr dazu siehe Unten)
  print(mock_response.choices[0].message.content)



Hi there, this is the mock response, not a real LLM response.


## Input Format



Bei einer API muss man die Daten klar strukturiert übergeben, damit das System versteht, was man möchte. In einem UI wie ChatGPT ist es so: Man gibt eine Frage im Textfeld ein (Input) und bekommt eine Antwort in dem Chatfenster (Output). Bei einer API passiert dasselbe, aber in einer festgelegten, maschienenlesbaren Struktur – meist im JSON-Format. Da wir die OpenAI Bibliothek verwenden senden wir die Daten nicht im JSON Format, sondern geben es über Argumente der `create` Funktion des `chat.completion` Endpoints. Das klingt nach viel Jargon, aber ich hoffe an dem Beispiel wird es ersichtlich.

Ich instanziiere eine Variable namens `client` mithilfe von OpenAI's Python Bibliothek.

```python
from openai import OpenAI
client = OpenAI(api_key=API_KEY)
```
In dieser variable sind nun die verschiedenen Endpoints der API verfügbar. Wir interessieren uns für Chat completion aber es gibt auch andere wie `client.audio.speech`. Nun wollen wir, dass eine Antwort "kreiert" wird, wofür die `create` methode benutz wird. Die Antwort wollen wir in einer Variable namens `response` speichern.
```python
response = client.chat.completions.create()
```

Die `create` Methode hat verschiedene Argumente, die gesamte Dokumentation findet sich [hier](https://platform.openai.com/docs/api-reference/chat/create). Zwei Argumente sind zwingend erforderlich (*required*):
- `messages`: Eine Liste des bisherigen Chat-Verlaufs.
- `model`: Welches Modell verwendet werden soll. OpenAI bietet verschiedene Modelle mit unterschiedlichen Grössen und Kapazitäten an. Eine aktuelle Liste findet sich [hier](https://platform.openai.com/docs/models#gpt-4o).

Der Chat-verlauf (*messages*) muss in einer Liste bereitgestellt werden. Jedes Element in der Liste ist ein Dict mit 2 einträgen ```"role" ``` und ```"content"```. Schauen wir uns ein Beispiel an:

```python
 messages = [
    {"role": "system", "content": "Du bist ein hilfreicher Assistent."},
    {"role": "user", "content": "Was ist eine API?"},
    {"role": "assistant", "content": "Eine API ist eine Schnittstelle, die es ermöglicht, dass Programme miteinander kommunizieren."},
    {"role": "user", "content": "Was ist ein API Endpoint?"}
  ]
  ```

Es wechseln sich also `"user"` und `"assistant"`ab. Von den Servern der API erwarten wir den nächsten Eintrag für den `"assistant"`. Der erste Eintrag bestimmt welche Rolle das Modell hat. Man könnte hier zum Beispiel sagen `"Du bist ein gelangweilter Tutor und gibst nur sehr kurze, kaum hilfreiche Antworten"` um den Stil der Antworten vom Modell zu beeinflussen. In neueren Modellen empfiehlt OpenAI diese erste Nachricht mit der Rolle `"developer"`zu bezeichnen.

### Aufgabe 1
Versuche nun selber einen API-Call zu schreiben indem du das Code-Beispiel von oben kopierst entsprechend anpasst mit neuen Prompts.

WICHTIG: Solange wir keinen gültigen API-Key angegeben haben wird sich die Antwort nicht verändern. Es lohnt sich aber trotzdem zu überlegen, was wir für Antworten erwarten würden.

## How to receive a response
Was passiert hier:
```python

        answer = response.choices[0].message["content"]
```


Wenn wir die API aufrufen bekommen wir (vorausgesezt wir haben einen gültigen API-Key) eine Antwort. Diese Antwort kommt im JSON format und wird, da wir die Python-Bibliothek verwenden, ein dictionary-ähnliches Objekt umgewandelt. Hier ein Beispiel, wie ein solches Objekt aufgebaut ist (ein verkürztes Beispiel):
```json
{
  "id": "chatcmpl-123456",
  "object": "chat.completion",
  "created": 1728933352,
  "model": "gpt-4o-2024-08-06",
  "choices": [
    {
    
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hi there! How can I assist you today?",
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ]
}
```


Die ersten vier Einträge (id, object, created, model) sind Metadaten zu dem Request, den wir gemacht haben. Was uns interessiert ist die Antwort des Modells. Den finden wir unter "choices". Dieser eintrag ist eine Liste, in diesem Fall mit nur einem Eintrag. Man kann beim senden des Requests spezifizieren, dass man mehrere Antworten möchte, dann wird das Modell mehr als einmal mit derselben Prompt eine Antwort generieren. In dem Beispiel hat es nur einen Eintrag (also an der Stelle `0`). Von diesem ersten Eintrag der Liste wollen wir nun die messages und deren Content. Das Completion Object kann man wie ein Python Dictionary abfragen, das heisst:
```python
response["choices"][0]["message"]["content"]
```
Alternativ kann man auch "dot notation" verwenden:
```python
response.choices[0].message.content
```

### Aufgabe 2

Wie bereits erwähnt habe ich im `helper` modul habe ich ein Objekt erstellt, das sich so verhalten sollte als wäre es eine response vom OpenAI API und es `mock_response` genannt. Für diese Aufgabe habe ich das Objekt `mock_response_exercise` erstellt. Dieses enthält zwei Einträge in der Choices Liste. Verändere die Code Zelle so, dass es den Content der zweiten Nachricht ausgibt.

In [3]:
from helper import mock_response_exercise
print(mock_response_exercise)

{
  "id": "chatcmpl-123",
  "object": "chat.completion",
  "created": 1700000000,
  "model": "gpt-4o",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Die Aufgabe ist fast richtig gel\u00f6st."
      },
      "logprobs": null,
      "finish_reason": "stop"
    },
    {
      "index": 1,
      "message": {
        "role": "assistant",
        "content": "Die Aufgabe ist richtig gel\u00f6st :D"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": null,
  "system_fingerprint": null
}


## Programmatisches Erstellen der Inputs

Nun haben wir gelernt, wie man die API verwendet. Aber bis jetzt gibt es uns eher Mehraufwand als wenn wir das UI im Browser verwenden. Der wahre Nutzen von APIs kommt erst zur Geltung, wenn wir mehrere ähnliche Anfragen stellen wollen.

Nemen wir folgende Problemstellung, die ihr mit Hilfe von einem Sprachmodell lösen wollt:
- In einem Ordner habt ihr eine Anzahl von Dateien mit Althochdeutschem Text. 
- Ihr möchtet gerne Versionen dieser Dateien anlegen, die für moderne Leser verständlich sind.
- Im Idealfall würdert ihr ein(e) Historiker(in) anstellen um Kontextgerechte und verlässliche Übersetzungen zu erhalten. Aber euer Budget ist klein und um sich einen Einblick zu gewähren entscheidet ihr ein Sprachmodell hinzuzuziehen.



Da wir ein bestimmtes Antwortformat möchten, ist es wichtig, dass wir dem Modell ein Beispiel geben, wie die Antwort aussehen soll. Wir wollen lediglich den übersezten Text, nicht zusätzlichen text wie "Hier ist eine Übersetzung des gegebenen Textes". Dafür senden wir eine Beispielantwort als gegebener Chat-Verlauf. <br>


In der folgenden Codezelle ist ein Beispiel Code für das oben beschriebene Problem. Ihr müsst nicht jedes Detail verstehen, versucht aber die grundsätzliche Logik anhand der Kommentare nachzuvollziehen. Wenn ihr denkt, dass ihr den Code einigermassen verstanden habt macht Euch an die Aufgabe 3.

In [ ]:
from openai import AuthenticationError  # Der Error, den wir abfangen wollen
import os  # os modul zum iterieren über Dateien in einem Ordner
from helper import mock_response

# Diesen Teil des Chatsverlaufs wollen wir immer mitsenden. 
# Er enthält die System-nachricht und ein Beispiel mit der korrekten Antwort.
default_messages = [
    {"role": "system", "content": "Du bist Historiker, spezialisiert auf das Mittelalter. Deine Aufgabe ist es Althochdeutsche Texte in modernes Hochdeutsch zu übersetzen"},
    {"role": "user", "content": """Übersetze in Modernes Deutsch: Unser früntliche dienst voran, fursichtigen, ersamen, wysen, bsunder lieben und guͦten frund. Es habent unsere gsanten, die uff nächstgehaltnem tag by uch zuͦ Basel gwesen sind2, unsb furgebracht, was desselbigen tags gehandeltc."""},
    {"role": "assistant", "content": """Unser freundlicher, ehrenhafter, vorsichtiger, gewissenhafter, weiser, besonders lieber und guter Freund. Unsere Gesandten, die beim zuletzt abgehaltenen Tag bei Euch in Basel gewesen sind, haben uns berichtet, was an jenem Tag verhandelt wurde."""}

]

# Eine for-Schleife über die dateinamen im Ordner "althochdeutsch"
for filename in os.listdir("althochdeutsch"):
    # Der Pfad jeder Datei ist althochdeutsch/beispiel_xyz.txt
    infilepath = os.path.join("althochdeutsch", filename)

    # öffne die Datei und speichere den Text in der Variable 'text'
    with open(infilepath, "r", encoding="utf-8") as infile:
       text = infile.read()

    # Wir fügen nun den Text der Datei unserer Prompt hinzu.
    # In der Rolle des 'user' geben wir den althochdeutschen Text als content
    current_file_query = [{"role": "user", "content": text}]
    # erstellen der Liste mit der neuen User query
    messages = default_messages + current_file_query

    # wie oben, rufen wir nun die API auf
    try: 
        response = client.chat.completions.create(  
            model="gpt-4o-mini",
            # wir setzen den vorher kreierten Chat-Verlauf hier ein
            messages=messages
        )
        # Die Response des Servers 
        answer = response.choices[0].message["content"]

    # Abfangen, wenn es einen AuthenticationError gibt 
    except AuthenticationError as e:
        print("Authentication failed, using mock response")
        # wenn wir keine Antwort haben 
        answer = mock_response.choices[0].message["content"]
    
    # Prompt speichern. Das ist in jedem Fall Sinvoll für potentielles Debugging oder Wiederholbarkeit der Experimente
    with open (os.path.join('prompts', filename), "w", encoding="utf-8") as outfile:
        for message in messages:
            outfile.write(f"{message}\n")

    # Die Antwort in einer datei speichern
    if answer is not None:  # Antwort speichern
        with open(os.path.join('hochdeutsch', filename), "w", encoding="utf-8") as outfile:
            outfile.write(answer)
    


Authentication failed, using mock response
Authentication failed, using mock response
Authentication failed, using mock response


## Aufgabe 3
Der Ordner "bad_xml" enthält XML Dateien mit Syntaxfehlern. Copiere den Code von Oben und verändere ihn so, dass wir das Sprachmodell beauftragen die Fehler zu korrigieren. Anschliessend soll der Code die Fertige Datei im ordner good_xml speichern. (Die Prompts können im selben Ordner gespeichert werden, wie die Übersetzungsprompts vom Beispiel)

## Strukturierte Outputs

LLMs verstehen nicht nur "natürliche Sprache", sondern auch Programmiersprachen und verschiedene Datenformate. Ich kann also einem LLM einen Task geben und spezifizieren, dass ich den Output gerne in einem gewissen Datenformat (zum Beispiel JSON) bekommen möchte. Die Natur von Sprachmodellen ist aber, dass sie nicht garantiert das richtige Format einhalten. Im Falle von JSON gibt uns die API von OpenAI die möglichkeit sicher zu stellen, dass die Antwort des Modells im JSON-Format erscheint.

Ein Beispiel Simon Climatides Kurs *Programmiertechniken der Computerlinguistik*<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1): Wir haben historische Texte des British Alpine Magazine und möchten eine Datenbank anlegen, in der wir notieren welche Alpinisten welche Berge bestiegen haben. Eine Person, müsste dazu alle Einträge im Archiv durchlesen und diese zum Beispiel in eine Excel-Sheet eintragen. Eine andere Option ist es diesen Task eine Sprachmodell zu geben. Dieses kann uns JSON dateien geben, die wir nacher problemlos mit anderer software weiterverwenden können. Wir könnten folgende Prompt formulieren:

<a name="cite_note-1"></a>[<sup>[1]</sup>](#cite_ref-1) Das Beispiel wurde erstellt zusammen mit den Tutoren: Kasper Powell , Marius Muller und Mert Erol

In [23]:
system_message = "You are an expert in mountaineering."
user_prompt = """Here is a paragraph from the OCR-ized British Alpine Magazine from 1969.

PARAGRAPH:
"The ranks ot ' famous ' climbers can never be closed .
Hut what does tame mean to climbing ?
The true climber seeks his fulfilment in the limitations set to him personally by the forces of Nature .
Whether this fulfilment is found in walking amongst mountains or in conquering extreme routes , it is all the same in the end .
With luck , climbing will never become a competitive ' Sport ' , nor appear in the Olympic Games ,
although already two Olympic Gold Medals have been bestowed to climbers one in iy}2 to the brothers Schmid
for their first ascent ot the Matterhorn North face and another in 10,^6 to Günther Oskar Dyhrenfurth for his success in
his great Himalayan undertakings .
"


Now, analyze and report for each person in a JSON data structure of the following format:

    {
      "mountain": MOUNTAIN,
      "relations":[
        {"person_name": PERSON_NAME1, "is_mountaineer": true/false/null , "climbed_mountain": true/false/null},
        {"person_name": PERSON_NAME2, "is_mountaineer": true/false/null , "climbed_mountain": true/false/null},
        ...
        ]
    }

Use your background knowledge and the paragraph to decide whether the recognized person is a mountaineer (true) or not (false).
In case you don't know, set the value to null.

For each mountaineer (person with is_mountaineer: true), decide whether there is textual evidence in the paragraph
that they actually climbed this mountain. Use the value null if the evidence in the paragraph is inconclusive.

"""

json_prompt_messages = [
      {"role": "system", "content": system_message},
      {"role": "user", "content": user_prompt}
    ]

Diese Prompt allein wird uns schon recht gute Resultate liefern. Wir wollen aber sicher gehen, dass wir immer korrektes JSON Datenformat bekommen, denn wir können uns nicht darauf verlassen, dass das Sprachmodell keine Fehler macht (zum Beispiel Anführungszeichen vergessen). Um den Output zu spezifizieren verändern wir den API Call, fügen `system_message` und `user_prompt` bei den messages ein und erweitern den API call um ein zusätzliches Argument names `response_format`. 

**Wichtig**: Wenn wir dieses Argument geben, ist es unerlässlich, dass die Prompt ein Beispiel enthält das zeigt wie wir den Output formatiert haben wollen. Sonst kann es zu Problemen kommen, nämlich dass das LLM eine Antwort in einem komplett anderen Format beginnt, während der Server versucht diesen in ein JSON format umzuwandeln. Das kann dazu führen, dass die Antwort unbrauchbar und ausserdem sehr lang (und damit teuer) wird.

In [ ]:
from openai import AuthenticationError  # Der Error, den wir abfangen wollen
from helper import mock_json_response

try:  # solange kein Error entsteht tue dies

  # Abfragen des Servers und die Antwort des Servers in der Variable 'response' speichern
  response = client.chat.completions.create(  
    model="gpt-4o-mini",
    # Fülle die Messages mit den Prompts von oben
    messages=json_prompt_messages,
    # Definition des Formats als JSON Objekt
    response_format={
      "type": "json_object"
    }
  )
  # Die Antwort aus der Response herausholen
  json_answer = response.choices[0].message.content

# Abfangen, wenn es einen AuthenticationError und eine Mock-Response verwenden
except AuthenticationError as e:
  print("There was an authentication error. Will be using the mock_json_response")
  json_answer = mock_json_response.choices[0].message.content

# die Prompt speichern
with open(os.path.join("prompts", "json_example_prompt.txt"), "w", encoding="utf-8") as outfile:
  for message in json_prompt_messages:
    outfile.write(f"{message}\n")

# Json Antwort speichern
with open("json_response.json", "w", encoding="utf-8") as outjson:
  outjson.write(json_answer)


There was an authentication error. Will be using the mock_json_response


Die Code Zelle gibt keinen direkten Output, die Antwort wird in `json_response.json` gespeichert. Ihr könnt dort nachschauen wie diese Aussieht.

## Aufgabe 4

Im Ordner `fussball_nachrichten` sind kurze Meldungen über Fussballspiele in drei kleinen Beispieldateien. Benutze den Code von der vorherigen Aufgabe mit dem Code vom Beispiel mit den Bergsteigern um eine Prompt für jede Datei zu schreiben, die das LLM damit beauftragt die Spielergebnisse in folgendem Format auszugeben: 
```json
{
  "spiele": [
    {
      "heimteam": "TEAM_NAME",
      "auswärtsteam": "TEAM_NAME",
      "ergebnis": "x:y"
    },
```
Wie oben, sollte es `null` angeben, wenn die Information nicht vollständig aus dem Text zu entnehmen ist. Die fertigen JSON Dateien sollten im Ordner `fussball_nachrichten_json` gespeichert werden.

**Tipp 1**:

Fokussiert euch zuerst auf die Prompt, das heisst macht das For-Loop mit nur der Prompt, und stellt sicher dass es richtig funktioniert.
Wir müssen eine lange Prompt erstellen, die sich ändert je nach Inputdatei. Dabei können wir wie folgt vorgehen:
1. Wir definieren die Teile der Prompt die immer gleich bleiben:


In [8]:
system_prompt = "Du bist Sportreporter in der Schweiz."
user_prompt_part1 = """finde alle Fussballspiele im Text und gib mir ein json mit den daten zu den Spielergebnissen

Hier der Text:

"""

user_prompt_part2 = """Gib mir die Antwort im folgenden JSON format:
{
  "spiele": [
    {
      "heimteam": "TEAM_NAME",
      "auswärtsteam": "TEAM_NAME",
      "ergebnis": "x:y"
    },
    ...
}

Wenn die Information im Text unvollständig ist, gebe "null" an."""

2. Innerhalb des For-Loops fügen wir den text aus der Datei in die Prompts ein:

In [28]:
user_prompt = user_prompt_part1 + text + user_prompt_part2
json_prompt_messages = [
      {"role": "system", "content": system_message},
      {"role": "user", "content": user_prompt}
    ]

**Tipp 2**:

Beachtet, dass ihr die Antworten nicht als `txt`-Datei sondern mit der file-extention `.json` speichert, damit andere Programme auch wissen welches Format sich dahinter befindet. Ihr könnt Dateinamen wie Strings verändern:

In [7]:
import os
for filename in os.listdir("fussball_nachrichten"):
    print("Originaldateiname:", filename)
    json_filename = filename.split(".")[0] + ".json" 
    print("JSON name:", json_filename)
    print()

Originaldateiname: nachricht1.txt
JSON name: nachricht1.json

Originaldateiname: nachricht2.txt
JSON name: nachricht2.json

Originaldateiname: nachricht3.txt
JSON name: nachricht3.json



In [ ]:
# Todo: Your code goes here ;D
